In [1]:
from Player import Player
from Battle import Battle
from Monster import Monster
import time, json
import pandas as pd
from openai import OpenAI
import os

print(os.getcwd())

c:\Users\okan2\Desktop\Seminar\Code\AgentsAndDragons


In [2]:
magePlayer = Player("Mage", 120, 50, 25, pd.read_excel("Abilities.xlsx"))
players = [magePlayer]
#monster = Monster("Slime", 200, 200, 10, weaknesses=["Lightning"])
monster = Monster("Slime", 150, 150, 20, weaknesses=["Fire"])


b = Battle(players, monster)
b.battlePrint

A Slime draws near!

Mage's HP: 120/120 | MP: 50/50

Slime's HP: 150/150

What will you do?


"A Slime draws near!\nMage's HP: 120/120 | MP: 50/50\nSlime's HP: 150/150\nWhat will you do?"

In [3]:
from apiKey import key

## AUTOGEN

In [4]:
from autogen.oai.openai_utils import retrieve_assistants_by_name
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.retrieve_utils import TEXT_FORMATS
from typing_extensions import Annotated

Agent Definitions

In [5]:
mageAgent = RetrieveAssistantAgent(
    name="Mage",
    llm_config={"config_list": [{"model": "gpt-3.5-turbo", "temperature": 1, "api_key": key}],
    },
    system_message="You are an AI assistant taking the role of a mage fighting a turn-based battle against an enemy."
    "You are fighting the battle with a partner warrior. He may ask you inbetween turns to discuss the next actions to take. In this case, "
    "try to elaborate on the situation and say what you think the best course of action would be."
    "Remember the advice your teammate gives you when choosing your next action."
    "While you are having a conversation with your partner about the enemy, only talk to each other, do not call any functions or tools. "
    
    "Outside of the conversation with your partner, you will interact with the battle in the following way:"
    "1. You will be presented with the state of the battle with the participants HP and MP values as well as everything that has happened previously."
    "2. When you are asked what you will do, you have to decide and choose the action you will enter for this turn."
    "3. First, call the function mageListAbilities to get a list of abilities available to you."
    "4. Then you will choose an ability from the list and call the function mageAbility with the name of the ability as the parameter."
    "Only call the function mageAbility once per turn."
    "The goal is to deal as much damage as possible to the enemy. Consider the damage multipliers and the attributes when choosing an ability."
    "Enemies can have weaknesses to certain attributes."
    "For example, if you know that a slime is weak to lightning, you should use Lightning Bolt instead of Fireball, even if Fireball has a higher multiplier."
    "If you have not encountered an enemy before, you do not know any weaknesses. Try out all of your abilities until you land a critical hit."
    "If the ability does not land a critical hit, you know that the enemy is not weak to the attribute of the ability you used. Try another ability in this case."
    "If you realize that one of your abilities has dealt critical damage to an enemy, you have discovered a new weakness of that enemy."
    "Be sure to remember it for the next turns and reuse it in the following turns."
    "Lay out your plan of action before executing it. Break down each step in your consideration."
    "Do not ask for any human input and execute all your decisions by calling functions every turn until the battle ends."
    "Do not ask questions while formulating your plan of action."
    "If you decide to call a function do not forget to execute the corresponding python code aswell."
    "If you run out of mana, you can either use another ability with lower mana costs or, if you cannot use any other abilities, use a simple attack with the function mageAttack, since a simple attack does not use any MP."
    "Instead of using an ability or attacking, you may also defend with the function mageDefend. This will protect you from some damage."
    "Repeat the above steps as long as the battle has not ended. The battle is only over if you receive a message about either Victory or Defeat."
    "There will be no termination until the enemy is defeated. Always move forward with your actions, do not wait and do not ask for any human input."
    "This means, if you decide to choose a specific ability, immediately call the corresponding function."
    #"Always continue the battle until it is over. Do not terminate until the battle is over."
    "When the battle ends, reply with TERMINATE."
)

In [6]:
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    default_auto_reply="Ok",
    max_consecutive_auto_reply=20,
    retrieve_config={
        "docs_path": [
            "Abilities.csv"
        ],
        "embedding_model": "all-mpnet-base-v2",
        "get_or_create": True,
        "last_n_messages" : 1
    },
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": os.getcwd(),
        "use_docker": False,
    },  # set to False if you don't want to execute the code
    
)

c:\Users\okan2\anaconda3\envs\AgentsAndDragons\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Mage Abilities

In [7]:
mageAbilityCounter = {"attack" : 0,
                      "abilityList" : 0,
                      "ability" : 0,
                      "defend" : 0
                      }

In [8]:

@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageListAbilities", description="Returns a dataframe with details of the mages abilities")
def mageListAbilities() -> pd.DataFrame:
    mageAbilityCounter["abilityList"] += 1
    return pd.read_csv("Abilities.csv", sep=";").to_json()

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageAbility", description="Execute the ability of the mage")
def mageAbility(name : Annotated[str, "Name of the used ability"]) -> str:
    b.enterAbility(magePlayer, name)
    mageAbilityCounter["ability"] += 1
    return b.battlePrint

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageAttack", description="Attack the enemy")
def mageAttack() -> str:
    b.enterAttack(magePlayer)
    mageAbilityCounter["attack"] += 1
    return b.battlePrint

# @group_chat_manager.register_for_execution()
@ragproxyagent.register_for_execution()
@mageAgent.register_for_llm(name="mageDefend", description="Defends against incoming attacks")
def mageAttack() -> str:
    b.enterDefend(magePlayer)
    mageAbilityCounter["defend"] += 1
    return b.battlePrint

Chat Initiation

In [9]:
chat = ragproxyagent.initiate_chat(mageAgent,
                                message=b.battlePrint,
                                max_turns=20,
                                summary_method="reflection_with_llm",
                                summary_args={"summary_prompt" : "Summarize the monster you fought and the weaknesses you discovered."}
                                )

print(chat)

ragproxyagent (to Mage):

A Slime draws near!
Mage's HP: 120/120 | MP: 50/50
Slime's HP: 150/150
What will you do?

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

***** Suggested tool call (call_rvqKheHwxCYmLxK7qG7p3SDB): mageListAbilities *****
Arguments: 
{}
**********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION mageListAbilities...


[runtime logging] log_function_use: autogen logger is None


ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_rvqKheHwxCYmLxK7qG7p3SDB) *****
{"Name":{"0":"Fireball","1":"Lightning Bolt","2":"Icecicle","3":"Dark Obliteration","4":"Inferno","5":"Magic Explosion"},"Description":{"0":"Shoots a small fireball at the enemy","1":"Zaps the enemy with the power of lightning","2":"Drops an icecicle on the enemy from above","3":"Conjures shadows to crush foes, inflicting heavy damage.","4":"Engulfs enemies in flames, causing widespread damage.","5":"Creates a huge explosion using the full power of the user's magic."},"Damage Multiplier":{"0":"1,10","1":"1,05","2":"1,10","3":"2,00","4":"1,50","5":"2,50"},"Attributes":{"0":"Fire","1":"Lightning","2":"Ice","3":"Dark","4":"Fire","5":"Light"},"Cost":{"0":5,"1":10,"2":8,"3":30,"4":20,"5":50}}
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

T

[runtime logging] log_function_use: autogen logger is None


Mage has selected Fireball

Mage's turn:Mage uses Fireball! Mage deals 36.300000000000004 critical damage to Slime! Slime is weak to Fire!
Monster's turn:Slime attacks! Slime deals 17 damage to Mage!

Mage's HP: 103/120 | MP: 45/50

Slime's HP: 113.69999999999999/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_OK9ST7J4Kjmnyp0Av6Ak6xzB) *****

Mage's turn:Mage uses Fireball! Mage deals 36.300000000000004 critical damage to Slime! Slime is weak to Fire!
Monster's turn:Slime attacks! Slime deals 17 damage to Mage!
Mage's HP: 103/120 | MP: 45/50
Slime's HP: 113.69999999999999/150
What will you do?
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

Great news! The Slime is weak to Fire. I will continue to exploit this weakness by using Fireball again. Let's keep the pressure on.

------

...

Mage has selected Fireball

Mage's turn:Mage uses Fireball! Mage deals 47.85 critical damage to Slime! Slime is weak to Fire!
Monster's turn:Slime attacks! Slime deals 21 damage to Mage!

Mage's HP: 82/120 | MP: 40/50

Slime's HP: 65.85/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_7L5U0D7s1sDHbxB4XSwxNz69) *****

Mage's turn:Mage uses Fireball! Mage deals 47.85 critical damage to Slime! Slime is weak to Fire!
Monster's turn:Slime attacks! Slime deals 21 damage to Mage!
Mage's HP: 82/120 | MP: 40/50
Slime's HP: 65.85/150
What will you do?
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

The Fireball is proving very effective against the Slime's weakness to fire. I will use Fireball again to take advantage of this weakness and deal more damage. Let's keep the momentum going.
***

[runtime logging] log_function_use: autogen logger is None


Mage has selected Fireball

Mage's turn:Mage uses Fireball! Mage deals 42.900000000000006 critical damage to Slime! Slime is weak to Fire!
Monster's turn:Slime attacks! Slime deals 18 damage to Mage!

Mage's HP: 64/120 | MP: 35/50

Slime's HP: 22.94999999999999/150

What will you do?
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_q43aINXfOgQPOeincXpXUteb) *****

Mage's turn:Mage uses Fireball! Mage deals 42.900000000000006 critical damage to Slime! Slime is weak to Fire!
Monster's turn:Slime attacks! Slime deals 18 damage to Mage!
Mage's HP: 64/120 | MP: 35/50
Slime's HP: 22.94999999999999/150
What will you do?
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

The Slime is still weak to Fire, so I will continue to exploit this weakness by using Fireball once more. Let's finish off the Slime with its we

[runtime logging] log_function_use: autogen logger is None


Mage has selected Fireball

Mage's turn:Mage uses Fireball! Mage deals 37.95 critical damage to Slime! Slime is weak to Fire!

Mage's HP: 64/120 | MP: 30/50

Slime's HP: 0/150


Victory! Mage defeated Slime!
Turn executed
ragproxyagent (to Mage):

ragproxyagent (to Mage):

***** Response from calling tool (call_6NRDbgMrKhOv9CGeP2ZOF0FX) *****

Mage's turn:Mage uses Fireball! Mage deals 37.95 critical damage to Slime! Slime is weak to Fire!
Mage's HP: 64/120 | MP: 30/50
Slime's HP: 0/150

Victory! Mage defeated Slime!
**********************************************************************

--------------------------------------------------------------------------------
Mage (to ragproxyagent):

TERMINATE

--------------------------------------------------------------------------------
ChatResult(chat_id=None, chat_history=[{'content': "A Slime draws near!\nMage's HP: 120/120 | MP: 50/50\nSlime's HP: 150/150\nWhat will you do?", 'role': 'assistant'}, {'tool_calls': [{'id': 'call_rvqKheHwx

## OPENAI API

In [10]:
client = OpenAI(api_key=key)

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)

mage = my_assistants.data[0]
print(mage)

Assistant(id='asst_ONJDOIMw7aK3PDIiB6aMX3Jm', created_at=1711721242, description=None, file_ids=['file-1GVa4WQad6GMFJh3lYwa12wd'], instructions='You are a mage fighting a turn-based battle against a randomly chosen enemy.\nYou have a list of abilities available to you found in the file "Abilities.xlsx".\nWhen you are asked what you will do, you will choose one ability from the file "Abilities.xlsx". Then you call the function useAttackAbility with the name of the ability.\nIf the function returns false, you used an ability that you do not possess. Check the file "Abilities.xlsx" again to see what you can use.\nIf the function returns true, wait for the next instruction from me and do not continue the battle on your own. For example you can just say "Ok, lets see how the battle turns out".', metadata={}, model='gpt-3.5-turbo-0125', name='Mage', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter'), RetrievalTool(type='retrieval'), FunctionTool(function=FunctionDefiniti

In [11]:
#start thread
thread = client.beta.threads.create()

#intialize moveList used for every turn
moveList = []

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=b.battlePrint
)

# while not b.is_over:
print("NEW TURN")
print(b.battlePrint)
moveList = []

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=mage.id
)

  
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1) # Wait for 1 second
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run)

a = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)["name"]
print(a, type(a))

# message = client.beta.threads.messages.create(
#   thread_id=thread.id,
#   role="user",
#   content=b.battlePrint
# )

NEW TURN

Mage's turn:Mage uses Fireball! Mage deals 37.95 critical damage to Slime! Slime is weak to Fire!
Mage's HP: 64/120 | MP: 30/50
Slime's HP: 0/150

Victory! Mage defeated Slime!
SyncCursorPage[Message](data=[Message(id='msg_wypsYtuvrXjCK7RDKs0AjM7x', assistant_id='asst_ONJDOIMw7aK3PDIiB6aMX3Jm', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Great job defeating the Slime! If you encounter another enemy and would like to engage in another battle, feel free to start a new turn.'), type='text')], created_at=1718913093, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Vo68QvGOG6TM8td2pTv1oMLp', status=None, thread_id='thread_5fKbWaLEhpAlgK9JJWLKOUvH'), Message(id='msg_T1maUggV1qTtV6r1OD1ZdUOA', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="\nMage's turn:Mage uses Fireball! Mage deals 37.95 critical damage to Slime! Slime is 

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

## TESTCODE

In [ ]:
current = b.nextTurn(["Lightning Bolt"])
print("CURRENT", current)

In [ ]:
b.enterAttack(warrior)

In [ ]:
b.enterAbility(magePlayer, "Fireball")